In [1]:
def get_insert_query(table_name,**kwargs):
    # get_insert_query(Song_Index ,song_name = 'Nector of Instruction', song_short_name ='noi', sloka_statues=1)
    # print(type(kwargs))
    comma = r', '    
    Entry_Field =   ', '.join(list(kwargs.keys())) 
    # print(Entry_Field) # >>> song_name, song_short_name, sloka_statues
    Entry_Value = ''
    
    for key, value in kwargs.items():
        if isinstance(value, str): # type(value) == 'str': 
            Entry_Value = Entry_Value + """'{}'{}""".format(str(value),comma)    
        else:
            Entry_Value = Entry_Value + """{}{}""".format(str(value),comma)    
            # Entry_Value +  str() + '=' + str(value) + comma
    Entry_Value = Entry_Value.rstrip(comma)
    # print(Entry_Value)  # >>> 'Nector of Instruction', 'noi', 1

    query = """INSERT INTO {} ({}) VALUES ({});""".format(table_name,Entry_Field, Entry_Value)        
    # >>> INSERT INTO SongIndex (song_name, song_short_name, sloka_statues) VALUES ('Nector of Instruction', 'noi', 1);

    return query




def get_update_query(table_name, **kwargs):
    query = "UPDATE {} SET ".format(table_name)

    last_key = list(kwargs.keys())[-1]
    if isinstance(kwargs[last_key], str):
        query_ending = " WHERE {}='{}'".format(last_key, kwargs[last_key])
    else:
        query_ending = " WHERE {}={}".format(last_key, kwargs[last_key])
    del kwargs[last_key]
    
    comma = r', '
    for key, value in kwargs.items():
        if isinstance(value, str):
            query =  """{} {}='{}'{} """.format(query, key,value,comma)  #  query + str(key) + '=' + str(value) + comma
        else:
            query =  """{} {}={}{} """.format(query, key,str(value),comma)  

    query = query.rstrip(comma) 
    
    query += query_ending
    
    # print(query)   # >>> UPDATE store SET quantity=10, price=15 WHERE item=10
    return query
    
# get_update_query('store', quantity=10, price=15, item='indian mango')
        # UPDATE store SET quantity=10, price=15 WHERE item='indian mango'
# get_update_query('store', quantity=10, price=15, item=10)
        # UPDATE store SET quantity=10, price=15 WHERE item=10

def get_delete_query(table_name, **kwargs):
    table_index = list(kwargs.keys())[0]
    query = "DELETE FROM {} WHERE {} = {};".format(table_name,table_index, kwargs[table_index] )
    return query


In [2]:
import os, sqlite3 

db_name = 'slokabase.db'
db_path = os.path.join(os.getcwd(),db_name)
table_name = 'SongIndex'
print(db_path)

        
class SongIndex():
    def __init__(self,db_path,table_name):
        self.db_path = db_path
        self.table_name = table_name
        self.db_connect = sqlite3.connect(db_path) # OperationalError: database is locked may occur to reduce try  <db.connections.close_all()>
                                                # https://www.geeksforgeeks.org/how-to-list-tables-using-sqlite3-in-python/?ref=lbp
    
    def create_table(self):
        # self.db_connect = sqlite3.connect(db_path)
        self.db_cursor = self.db_connect.cursor()
        
        self.db_cursor.execute("""CREATE TABLE  if not exists  SongIndex (
                                    	song_idx INTEGER PRIMARY KEY , 
                                    	song_name VARCHAR(100) NOT NULL UNIQUE, 
                                    	song_short_name VARCHAR(100) NOT NULL UNIQUE, 
                                    	sloka_statues BOOLEAN NOT NULL CHECK (sloka_statues IN (0, 1)),                                     
                                    	division VARCHAR(100), 
                                    	division_no INTEGER, 
                                    	total_slokas INTEGER, 
                                        groups VARCHAR(100), 
                                    	devotion_god VARCHAR(100), 
                                    	author VARCHAR(100), 
                                    	describtion TEXT, 
                                    	other_links TEXT
                                        )""")
        self.db_connect.commit()
        # auto_increment and delete on cascade https://stackoverflow.com/questions/29037793/sqlite-integrityerror-unique-constraint-failed
        self.db_cursor.close()
        # self.db_connect.close()

    def create_entry(self, **kwargs):
        # print(kwargs)
        # print(get_insert_query(**kwargs))
        query = get_insert_query(self.table_name,**kwargs)
        # source code : https://sqlite.oDELETE FROM COMPANY WHERE ID = 7;rg/forum/info/fa9e5bf90aed1d720bae5428b307bbfff0b1b3b98ec71643f1d98c3ed3e2bc5a
        # self.db_connect = sqlite3.connect(db_path)
        self.db_cursor = self.db_connect.cursor()
        # query = """INSERT INTO SongIndex ( "song_name", "song_short_name", "sloka_statues")
        #             VALUES ( 'Nector of Instruction', 'noi', 0);""" # .formate
        try :
            self.db_cursor.execute(query)        
            self.db_connect.commit()
            # self.db_cursor.execute("""INSERT INTO SongIndex (song_name, song_short_name, sloka_statues)VALUES (?,?,?);""", 
            #                        ( 'Nector of Instruction', 'noi', 0))                    
            # raise IntegrityError
        except sqlite3.IntegrityError as e:
            print('Exception: {}'.format(e))

        finally:
            self.db_cursor.close()      
            # self.db_connect.close()
            # self.db.connections.close_all()
    
    def read_entry(self):
        query = "SELECT * FROM {};".format(self.table_name)
        self.db_cursor = self.db_connect.cursor()
        try :
            self.db_cursor.execute(query)
            rows = self.db_cursor.fetchall()

            for row in rows:
                print(row)

        except Error as e:
            print('Exception: {}'.format(e))
        finally:
            self.db_cursor.close()

    def update_entry(self, **kwargs):
        query = get_update_query(self.table_name,**kwargs)
        self.db_cursor = self.db_connect.cursor()
        print(query)
        try :
            self.db_cursor.execute(query)        
            self.db_connect.commit()
        except sqlite3.Error as e:
            print('Exception: {}'.format(e))

        finally:
            self.db_cursor.close()

    def delete_entry(self, **kwargs):
        self.db_cursor = self.db_connect.cursor()
        self.db_connect.commit()
    
        query = get_delete_query(self.table_name, **kwargs)
        print(query)
        try :
            self.db_cursor.execute(query)        
        except sqlite3.Error as e:
            print('Exception: {}'.format(e))

        finally:
            self.db_cursor.close()
            
    def db_close(self):
        self.db_connect.close()
        
    


/media/karthik/myVolume/workspace/workspae_vedabase/slokabase.db


In [3]:
mySlokaIndex = SongIndex(db_path,table_name) # Connect to Table inside a given db 

In [4]:
 
mySlokaIndex.create_table() # Create table
mySlokaIndex.create_entry(song_name = 'Nector of Instruction', song_short_name ='noi', sloka_statues=1,division_no=0)
mySlokaIndex.create_entry(song_name = 'Brahma-saṁhitā', song_short_name ='bs', sloka_statues=1,division_no=0)
mySlokaIndex.create_entry(song_name = 'Śrī Īśopaniṣad', song_short_name ='iso', sloka_statues=1,division_no=0)
# Śrī Īśopaniṣad

In [5]:
mySlokaIndex.create_entry(song_name = 'Nector of Instruction', song_short_name ='noi', sloka_statues=1,division_no=0)

Exception: UNIQUE constraint failed: SongIndex.song_short_name


In [6]:
mySlokaIndex.read_entry()

(1, 'Nector of Instruction', 'noi', 1, None, 0, None, None, None, None, None, None)
(2, 'Brahma-saṁhitā', 'bs', 1, None, 0, None, None, None, None, None, None)
(3, 'Śrī Īśopaniṣad', 'iso', 1, None, 0, None, None, None, None, None, None)


In [7]:
mySlokaIndex.update_entry(song_name = 'Nector of Instruction 2', song_short_name ='noi', sloka_statues=1, song_idx=1)
mySlokaIndex.read_entry()

UPDATE SongIndex SET  song_name='Nector of Instruction 2',   song_short_name='noi',   sloka_statues=1 WHERE song_idx=1
(1, 'Nector of Instruction 2', 'noi', 1, None, 0, None, None, None, None, None, None)
(2, 'Brahma-saṁhitā', 'bs', 1, None, 0, None, None, None, None, None, None)
(3, 'Śrī Īśopaniṣad', 'iso', 1, None, 0, None, None, None, None, None, None)


In [8]:
mySlokaIndex.delete_entry(song_idx=1)
mySlokaIndex.read_entry()

DELETE FROM SongIndex WHERE song_idx = 1;
(2, 'Brahma-saṁhitā', 'bs', 1, None, 0, None, None, None, None, None, None)
(3, 'Śrī Īśopaniṣad', 'iso', 1, None, 0, None, None, None, None, None, None)


In [9]:
mySlokaIndex.create_entry(song_name = 'Nector of Instruction', song_short_name ='noi', sloka_statues=1,division_no=0)
mySlokaIndex.read_entry()

(2, 'Brahma-saṁhitā', 'bs', 1, None, 0, None, None, None, None, None, None)
(3, 'Śrī Īśopaniṣad', 'iso', 1, None, 0, None, None, None, None, None, None)
(4, 'Nector of Instruction', 'noi', 1, None, 0, None, None, None, None, None, None)


In [7]:
mySlokaIndex.create_entry(song_name = 'Radhika Astakam', song_short_name ='radastk', sloka_statues=1,division_no=0,author='Krishnadasa Kaviraja Goswami')
mySlokaIndex.read_entry() 

(1, 'Nector of Instruction', 'noi', 1, None, 0, None, None, None, None, None, None)
(2, 'Brahma-saṁhitā', 'bs', 1, None, 0, None, None, None, None, None, None)
(3, 'Śrī Īśopaniṣad', 'iso', 1, None, 0, None, None, None, None, None, None)
(4, 'Radhika Astakam', 'radastk', 1, None, 0, None, None, None, 'Krishnadasa Kaviraja Goswami', None, None)


In [8]:
mySlokaIndex.db_close()